In [1]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import json
import ast

alt.data_transformers.enable("default", max_rows=None)
alt.renderers.enable("mimetype")
alt.theme.enable("fivethirtyeight")

ThemeRegistry.enable('fivethirtyeight')

## Datasets & Cleaning

### Battlefield

In [11]:
b4_data = pd.read_csv("../assets/SteamDB Battlefield 4.csv")
bh_data = pd.read_csv("../assets/SteamDB Battlefield Hardline.csv")
b1_data = pd.read_csv("../assets/SteamDB Battlefield 1.csv")
bv_data = pd.read_csv("../assets/SteamDB Battlefield V.csv")
b2042_data = pd.read_csv("../assets/SteamDB Battlefield 2042.csv")

In [12]:
b4_data.columns = b4_data.columns.str.lower()
bh_data.columns = bh_data.columns.str.lower()
b1_data.columns = b1_data.columns.str.lower()
bv_data.columns = bv_data.columns.str.lower()
b2042_data.columns = b2042_data.columns.str.lower()

In [13]:
b4_data[['peak', 'gain', 'average']] = b4_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
bh_data[['peak', 'gain', 'average']] = bh_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
b1_data[['peak', 'gain', 'average']] = b1_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
bv_data[['peak', 'gain', 'average']] = bv_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
b2042_data[['peak', 'gain', 'average']] = b2042_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)

fields_to_keep = ['month', 'peak', 'gain', 'average']
b4_data = b4_data[fields_to_keep][1:]
bh_data = bh_data[fields_to_keep][1:]
b1_data = b1_data[fields_to_keep][1:]
bv_data = bv_data[fields_to_keep][1:]
b2042_data = b2042_data[fields_to_keep][1:]

b4_data["month"] = pd.to_datetime(b4_data["month"], format="%b-%y")
b4_data = b4_data.sort_values("month")
bh_data["month"] = pd.to_datetime(bh_data["month"], format="%b-%y")
bh_data = bh_data.sort_values("month")
b1_data["month"] = pd.to_datetime(b1_data["month"], format="%b-%y")
b1_data = b1_data.sort_values("month")
bv_data["month"] = pd.to_datetime(bv_data["month"], format="%b-%y")
bv_data = bv_data.sort_values("month")
b2042_data["month"] = pd.to_datetime(b2042_data["month"], format="%b-%y")
b2042_data = b2042_data.sort_values("month")

In [ ]:
b_merged = pd.concat([b4_data, bh_data, b1_data, bv_data, b2042_data], ignore_index=True)
b_merged = b_merged[fields_to_keep]
b_merged.head()

,month,peak,gain,average
0,2020-06-01,1213.0,0.0,0.0
1,2020-07-01,1112.0,101.0,0.0
2,2020-08-01,3266.0,2154.0,0.0
3,2020-09-01,1119.0,2147.0,0.0
4,2020-10-01,5881.0,4762.0,0.0


In [15]:
SteamDB_b_merged_grouped = b_merged.groupby(b_merged.columns[0]).sum()

SteamDB_b_merged_grouped = SteamDB_b_merged_grouped.reset_index()
SteamDB_b_merged_grouped = SteamDB_b_merged_grouped.sort_values("month")
SteamDB_b_merged_grouped

,month,peak,gain,average
0,2020-06-01,5172.0,0.0,0.0
1,2020-07-01,4696.0,876.0,0.0
2,2020-08-01,16020.0,11324.0,0.0
3,2020-09-01,7273.0,8747.0,0.0
4,2020-10-01,15854.0,8581.0,0.0
...,...,...,...,...
58,2025-04-01,52747.0,28341.0,0.0
59,2025-05-01,59146.0,6399.0,0.0
60,2025-06-01,45532.0,13614.0,0.0
61,2025-07-01,44634.0,2654.0,0.0


In [ ]:
# b4_data.to_csv("../assets/SteamDB_b4_cleaned.csv",encoding="utf-8")
# bh_data.to_csv("../assets/SteamDB_bh_cleaned.csv",encoding="utf-8")
# b1_data.to_csv("../assets/SteamDB_b1_cleaned.csv",encoding="utf-8")
# bv_data.to_csv("../assets/SteamDB_bv_cleaned.csv",encoding="utf-8")
# b2042_data.to_csv("../assets/SteamDB_b2042_cleaned.csv",encoding="utf-8")
# SteamDB_b_merged_grouped.to_csv("../assets/SteamDB_b_merged_cleaned.csv",encoding="utf-8")